In [1]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [2]:
print("Total number of charaters",len(raw_text))

Total number of charaters 20479


In [3]:
print(raw_text[:99])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


#####   Our goal is to tokkenize this 20479 characters into individual word and special characters that we can then turn into embedding for LLM training

### Example of how splitting show be done 

In [4]:
import re 


text="Hello , LLMs. Split this sentence."
result=re.split(r'(\s)',text)

print(result)

['Hello', ' ', ',', ' ', 'LLMs.', ' ', 'Split', ' ', 'this', ' ', 'sentence.']


#### whenever there whitespace it split there
####   and we also we , . as token the 

In [5]:
result=re.split(r'([,.] | \s)', text)
result

['Hello ', ', ', 'LLMs', '. ', 'Split this sentence.']

#### we dont need whitespace token (if required like for llm for python code then we include it )

In [6]:
result=[item for item in result if item.strip()]
print(result)

['Hello ', ', ', 'LLMs', '. ', 'Split this sentence.']


#### we also need : ? ! as token 

In [ ]:
text="This is new--text ? ?"

In [14]:
result=re.split(r'([,.:;?"_!\']|--|\s)', text)
result=[item for item in result if item.strip()]
result

['This', 'is', 'new', '--', 'text', '?', '?']

#### applying this on our data

In [19]:
preprocessed=re.split(r'([,.:;?"_!\']|--|\s)', raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [20]:
print(len(preprocessed))

4685


## Step 2 creating Token IDs

#### we sort the token to determine the vocabulary size

In [21]:
all_words=sorted(set(preprocessed))
vocab_size=len(all_words)

print(vocab_size)

1131


In [22]:
vocab={token:integer for integer,token in enumerate(all_words)}

In [23]:
for i , item in enumerate(vocab.items()):
    print(item)
    if i>=50:
        break

('!', 0)
('"', 1)
("'", 2)
('(I', 3)
('(Though', 4)
(')', 5)
(',', 6)
('--', 7)
('.', 8)
(':', 9)
(';', 10)
('?', 11)
('A', 12)
('Ah', 13)
('Among', 14)
('And', 15)
('Are', 16)
('Arrt', 17)
('As', 18)
('At', 19)
('Be', 20)
('Begin', 21)
('Burlington', 22)
('But', 23)
('By', 24)
('Carlo', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Croft)', 30)
('Destroyed', 31)
('Devonshire', 32)
('Don', 33)
('Dubarry', 34)
('Emperors', 35)
('Florence', 36)
('For', 37)
('Gallery', 38)
('Gideon', 39)
('Gisburn', 40)
('Gisburns', 41)
('Grafton', 42)
('Greek', 43)
('Grindle', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)


#### based on output above the dictionary contain s individual token associated with unique integer labels

In [26]:
class SimpleTokenizerV1:

    def __init__(self, vocab):
        self.str_to_int =vocab
        self.int_to_str={i:s for s,i in vocab.items()}


    def encode(self ,text):
        preprocessed=re.split(r'([,.:;?"_!\']|--|\s)',text)

        preprocessed= [item.strip() for item in preprocessed if item.strip()]

        ids=[self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):

        text=" ".join([self.int_to_str[i]] for i in ids)

        text=re.sub(r'([,.:;?"_!\']|--|\s)',text)
        return text

In [ ]:
tokenizer=SimpleTokenizerV1(vocab)

text1=""""It's the last he painted, you know," Mrs. Gisburn said with pardonable pride"""

ids=tokenizer.encode(text1)
print(ids)

KeyError: 'wit'